In [ ]:
#!/usr/bin/python
# -*- coding: utf-8
#
# Copyright 2017 Ian Dobbie (ian.dobbie@bioch.ox.ac.uk)
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.
#
#
#routines to fit Lisa's SIM data with torridal Gaussians.

#which dataset to fit?
DATASET='20170110_eAna2CT'

#pixel sizes in um
LMPIXELSIZE=0.020
#to calculate FWHM from Sigma 
SIGMA2FWHM=2.35
#size of subimages to generate - with Lisa's data 8-10 seems sensible.
WINDOW=15

#is this really fixed?

#general useful imports
import numpy as N
from numpy import pi
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mppatches
import math
import copy
import time
import Mrc
import tifffile



from scipy.optimize import curve_fit
from gaussfitter import gaussfit
from gaussfitter import twodgaussian
from gaussfitter import mpfit
%matplotlib inline 



def ring_gaussian(params, shape=None):
    #radiusMajor,radiusMinor,angle,
    """
    Returns a 2d function that is a gaussian ring with major and minor axis and angle between them
    
    
    Parameters
    ----------
    shape : tuple
        if shape is set (to a 2-parameter list) then returns an image with the
        gaussian defined by inpars
    """
    if (len(params)==6):
        ellipse=False
        (centreX,centreY,radius,width,amp,background)=params
    elif (len(params)==8):
        ellipse=True
        (centreX,centreY,radiusMajor,radiusMinor,angle,width,amp,background)=params
    else:
        print "The ring_gaussian function needs 6 or 8 parameters"
        return False
        
    #distance from centre is r=((centreX-x)^2+(centreY-y)^2)^0.5
    #round gaussian is then g=b+a*exp(-(radius-r)/width^2)
    
    centreX=float(centreX)
    centreY=float(centreY)
    width=float(width)
    amp=float(amp)
    background=float(background)
    if ellipse:
        a=float(radiusMajor)
        b=float(radiusMinor)
        angle=float(angle)
    else:
        radius=float(radius)
    def ringGauss(x, y):
        
        if ellipse:
            #general ellipse is X(t)=a.cos(t) and Y(t)=b.sin(t) for angle t.
            # t = arctan(x/y)
            # then radius at angle t is (a*b)/(((a*a*(sin(t)^2.))+(b*b*(cos(t)^2.)))^0.5)
            # use numpy trig functions as x/y might be lists
            x=N.array(x,dtype=N.float)
            y=N.array(y,dtype=N.float)
            cx=x-centreX
            cy=y-centreY
            theta=N.arctan2(cy,cx)
            g=background+amp*N.exp(-((
                            ((((cx**2.0)+(cy**2.0))**0.5)-
                             ((a*b)/(((a*a*(N.sin(theta+angle)**2.))+(b*b*(N.cos(theta+angle)**2.)))**0.5)))
                               /width)**2.0))
        else:
            g=background+amp*N.exp(-(((((((x-centreX)**2.0)+((y-centreY)**2.0))**0.5)-radius)
                               /width)**2.0))
#        g=g+float(amp)*N.exp(-(((((float(centreX-x))**2.0+float((centreY-y))**2.0)**0.5)-radius)
#                               /(float(width)**2.0)))
        return g
    
    if shape is not None:
        return ringGauss(*N.indices(shape))
    else:
        return ringGauss


def ring_gfit(data, params, return_error=False, returnfitimage=False, **kwargs):
    """
    Fit a 2d circular ring Gaussian with starting params of the form:
    centreX,centreY,radius,width,amp,background
    
    return_error : bool
    Default is to return only the Gaussian parameters.
    If ``True``, return fit params & fit error
    returnfitimage : bool
    returns (best fit params,best fit image)
    
    
    Returns
    -------
    (params, [parerr], [fitimage]) | (mpfit, [fitimage])
    parameters : list
        The default output is a set of ring Gaussian parameters with the same shape
        as the input parameters
    fitimage : `numpy.ndarray`
        If returnfitimage==True, the last return will be a 2D array holding the
        best-fit model
    """
    data = data.view(N.ma.MaskedArray)
    err=1.0    
    
        
    def mpfitfun(data, err):
        def f(p, fjac):   
            ringg = ring_gaussian(p)
            delta = (data - ringg(*N.indices(data.shape))) / err
            return [0, delta.compressed()]
        return f
#centreX,centreY,radius,width,amp,background, shape=None):
#{'n': 1, 'value': params[1], 'limits': [minpars[1], maxpars[1]],
#                'limited': [limitedmin[1], limitedmax[1]], 'fixed': fixed[1],
#                'parname': "AMPLITUDE", 'error': 0},
    parinfo=[{'n': 0, 'value': params[0],'limits':[0,WINDOW*2.],'limited':[True,True],'parname':'yCentre','error': 0}]
 
    parinfo=parinfo+[{'n': 1, 'value': params[1],'limits':[0.,WINDOW*2],'limited':[True,True],'parname':'xCentre','error': 0}]
    parinfo=parinfo+[{'n': 2, 'value': params[2],'limits':[3.0,9.0],'limited':[True,True],'parname':'radiusMajor','error': 0}]
    parinfo=parinfo+[{'n': 3, 'value': params[3],'limits':[3.0,9.0],'limited':[True,True],'parname':'radiusMinor','error': 0}]
    parinfo=parinfo+[{'n': 4, 'value': params[4],'limits':[-N.pi,N.pi],'limited':[False,False],'parname':'angle','error': 0}]
    parinfo=parinfo+[{'n': 5, 'value': params[5],'parname':'width','error': 0}]
    parinfo=parinfo+[{'n': 6, 'value': params[6],'limits':[0,15000],'limited':[True,False],'parname':'amplitude','error': 0}]
    parinfo=parinfo+[{'n': 7, 'value': params[7],'limits':[0,2000],'limited':[True,False],'parname':'background','error': 0}]
    
  
    mp = mpfit.mpfit(mpfitfun(data, err), parinfo=parinfo,quiet=True)
    mpp = mp.params
    mpperr = mp.perror
    chi2 = mp.fnorm

    returns = mp.params
    if returnfitimage:
        fitimage = ring_gaussian(mp.params)(*N.indices(data.shape))
        returns = (returns, fitimage)
    return returns

def showplots(image,subImage,fitImages):
    fig = plt.figure()
    plt.subplot(131)
    plt.imshow(fitImages[image])
    plt.subplot(132)
    plt.imshow(subImage[image])
    plt.subplot(133)
    plt.imshow(subImage[image]-fitImages[image])
    plt.show()
    
def showplotfit(image,subImage,fitImages,fit):
    # Create a figure. Equal aspect so circles look circular
    fig,ax = plt.subplots(1)
    ax.set_aspect('equal')    
    # add the smlm image
    ax.imshow(subImage[image])
    #turn of autoscale to stop weird white borders.
    ax.autoscale(False)
    #plot red cross to show where we started. 
    ax.plot(fit[1],fit[0],'ro')
    # Show the image
    plt.show()

def printParams(params,res):
    #(centreX,centreY,radiusMajor,radiusMinor,angle,width,amp,background)
    print "centre= ",params[0],params[1]
    print "radii= ",params[2]*res,params[3]*res
    print "eccentricity= ", params[2]/params[3]
    print "angle= ", params[4]
    print "width= ", params[5]*res
    print "amplitude= ", params[6]
    print "background= ", params[7]
#Functions to plot images from localisation data from fastSPDM
#
def gaussian2d (centre_x,centre_y,width_x,width_y,shape):
    def gauss2d(x,y):
        #loop over each gaussian as defined by length a
        g=(1/(((width_x)+float(width_y))**0.5))*N.exp(-(((float(centre_x)-x)/float(width_x))**2 +
                                      ((float(centre_y)-y)/float(width_y))**2)/2.0)
        return g
    
    
    return gauss2d(*N.indices(shape))
def aveweight(xloc,yloc):
    return(((xloc**2)+(yloc**2))**0.5)
    
    
#function to minimize diff between fited torii and localisation positions.
#inital fit is just shift in XY.

def shiftfit(data1,data2, params, return_error=False, **kwargs):
    """
    Calulates a fit by least squares between two coordinate sets. 
    data1 and data2 must have the same shape. 
    return_error : bool
    Returns
    -------
    (params, [parerr], [fitimage]) | (mpfit, [fitimage])
    parameters : [xshift, yshift], [scale, angle]
        The default output is a vector of best fit params in the same order as the 
        input set
    """
    #eliminate possible NaN errors.
    data1 = data1.view(N.ma.MaskedArray)
    data2 = data2.view(N.ma.MaskedArray)

    err=1.0    
        
    def mpfitfun(data1,data2, err):
        def f(p, fjac):
            delta=(N.sum((data1-transPoint(data2,p))**2,axis=1))/err
            return [0,delta.compressed()]
        return f
    parinfo=[{'n': 0, 'value': params[0],'parname':'xshift','error': 0}]
 
    parinfo=parinfo+[{'n': 1, 'value': params[1],'parname':'yshift','error': 0}]
    parinfo=parinfo+[{'n': 2, 'value': params[2],'parname':'scale','error': 0}]
    parinfo=parinfo+[{'n': 3, 'value': params[3],'parname':'angle','error': 0}]
      
    mp = mpfit.mpfit(mpfitfun(data1,data2, err), parinfo=parinfo,quiet=True)
    mpp = mp.params
    mpperr = mp.perror
    chi2 = mp.fnorm

    returns = mp.params
    return returns

def transPoint(pointXY,t ):
    output=N.zeros((pointXY.shape))
    shifted=pointXY-t[:2]
    scaled=shifted*t[2]
    r=(scaled[:,0]**2+scaled[:,1]**2)**0.5
    theta=N.arctan2(scaled[:,1],scaled[:,0])
    output[:,0]=r*N.cos(theta+t[3])
    output[:,1]=r*N.sin(theta+t[3])
    return(output)

def invtransPoint(pointXY, t):
    output=N.zeros((pointXY.shape))
    #calculate r, theta transform
    r=(pointXY[:,0]**2+pointXY[:,1]**2)**0.5
    theta=N.arctan2(pointXY[:,1],pointXY[:,0])
    #retransfrom to cartisian with angular shift
    output[:,0]=r*N.cos(theta-t[3])
    output[:,1]=r*N.sin(theta-t[3])
    #divide by scale factor
    output=output/t[2]
    #shift back in XY
    output=output+t[:2]
    return(output)

#functions to transform between locations in sim pimage, localisation positions
#and pixels in localisation images
def sim2lmpos(pointXY,fit):
    return (invtransPoint(pointXY,fit))
def lm2simpos(pointXY,fit):
    return (transPoint(pointXY,fit))
def sim2lmpixelpos(pointXY,fit):
    return(sim2lmpos(pointXY,fit)/LMPIXELSIZE)
def lmpixel2simpos(pointXY,fit):
    return(lm2simpos(pointXY*LMPIXELSIZE,fit))
def lmpixel2lmpos(pointXY,fit=None):
    return(pointXY*LMPIXELSIZE)
def lmpos2lmpixel(pointXY,fit=None):
    return(pointXY/LMPIXELSIZE)

#this section deals with the SMLM data from fastSPDM
#general useful imports done above....

#Filter the points to those within a set distance of an XY position 
def filterPoints(x,y,distance,accuracy,data):
    #centre x,y
    #find point within distance 
    #eliminate points below accuracy
    #data is the fastSPDM input localisations
    output=N.zeros((data.shape))
    i=0
    for line in data:
        sep=checkdist(x,y,line[1],line[2])
        acc=(line[3]**2+line[4]**2)**0.5
        if (sep< distance):
            if (acc<accuracy):
                output[i]= line
                i=i+1
    return output[0:i,:]

#check distance between two points
def checkdist(x1,y1,x2,y2):
    return ((((x1-x2)**2)+((y1-y2)**2))**0.5)

def weightedMean(data,weights):
    #returns a weighted mean and variance
    wmean=N.sum(data*weights)/N.sum(weights)
    wvar=N.sum(weights*((data-wmean)**2))/N.sum(weights)
    return (wmean,wvar)

def fitcentre(data, params, return_error=False):
    """
    Iteratively adjust the centre point of a ring to minimise the
    stddev of the radii of a group of localisations from the ring
    data is inpout locations as a [[x1,y1],[x2,y2]...]
    params is the initial guess position
    """
    data = data.view(N.ma.MaskedArray)
    err=1.0    
        
    def mpfitfun(data, err):
        def f(p, fjac):
            radii=checkdist(p[0],p[1],data[:,0],data[:,1])
            delta=N.zeros((2))
            delta[0] = radii.std()
            delta[1] = radii.std()
            return[0,delta]
        return f
    
    parinfo=[{'n': 0, 'value': params[0],'parname':'xCentre','error': 0}]
    parinfo=parinfo+[{'n': 1, 'value': params[1],'parname':'yCentre','error': 0}]
    
    mp = mpfit.mpfit(mpfitfun(data, err), parinfo=parinfo,quiet=True)
    mpp = mp.params
    mpperr = mp.perror
    chi2 = mp.fnorm

    returns = mp.params
    
    return returns

#as fit centre above but uses the weightedMean fucntion to fit 
# by minimizing the weighted Variance. 
def fitweightedcentre(data, weights, params, return_error=False):
    """
    Iteratively adjust the centre point of a ring to minimise the
    weigthed variance of the radii of a group of localisations from the ring
    data is input locations as a numpy array [[x1,y1],[x2,y2]...]
    weights is the relative weights (1/localisation precision usually)
    params is the initial guess position
    """
    data = data.view(N.ma.MaskedArray)
    weights = weights.view(N.ma.MaskedArray)
    err=1.0    
        
    def mpfitfun(data, err):
        def f(p, fjac):
            radii=checkdist(p[0],p[1],data[:,0],data[:,1])
            (wmeanradii,wvarradii)=weightedMean(radii,weights)
#            print wmeanradii,wvarradii
            #hack as mpfit requires a list of output values to minimize.
            delta=N.zeros((2))
            delta[0] = wvarradii
            delta[1] = wvarradii
            return[0,delta]
        return f
    
    parinfo=[{'n': 0, 'value': params[0],'parname':'xCentre','error': 0}]
    parinfo=parinfo+[{'n': 1, 'value': params[1],'parname':'yCentre','error': 0}]
    
    mp = mpfit.mpfit(mpfitfun(data, err), parinfo=parinfo,quiet=True)
    mpp = mp.params
    mpperr = mp.perror
    chi2 = mp.fnorm

    returns = mp.params
    
    return returns

def plotsmlmregion(smlmimage,start,region):
    size=smlmimage.shape
    # Create a figure. Equal aspect so circles look circular
    fig,ax = plt.subplots(1)
    ax.set_aspect('equal')    
    # add the smlm image
    ax.imshow(smlmimage)
    # add circle of selected region
    selected=mppatches.Arc((size[0]/2.0,size[1]/2.0),region,region)
    ax.add_patch(selected)
    #turn of autoscale to stop weird white borders.
    ax.autoscale(False)
    #plot red cross to show where we started. 
    ax.plot(start[0],start[1],'ro')
    # Show the image
    plt.show()

def showmerged(mergedimage,simpos=None,lmpos=None,simfit=None,lmfit=None, region=None):
    size=mergedimage.shape
    centre =(size[0]/2.0,size[1]/2.0)
    # Create a figure. Equal aspect so circles look circular
    fig,ax = plt.subplots(1)
    ax.set_aspect('equal')    
    # add the smlm image
    ax.imshow(mergedimage)
    ax.autoscale(False)
    # add circle of selected region
    if (region and lmfit):
        selected=mppatches.Arc((lmfit[1]+centre[1],lmfit[0]+centre[0]),region,region)
        ax.add_patch(selected)
    if (simpos is not None):
        ax.plot(simpos[1]+centre[1],simpos[0]+centre[0],'ro')
    if (lmpos is not None):
        ax.plot(lmpos[1]+centre[1],lmpos[0]+centre[0],'bo')
    if (simfit is not None):
        ax.plot(simfit[1]+centre[1],simfit[0]+centre[0],'go')
    if (lmfit is not None):
        ax.plot(lmfit[1]+centre[1],lmfit[0]+centre[0],'ko')
        
    #turn of autoscale to stop weird white borders.

    #plot red cross to show where we started. 
#    ax.plot(start[0],start[1],'ro')
    # Show the image
    plt.show()
    
def showimage(image,meanfits=None):
#    print meanfits
    size=image.shape
    fig,ax=plt.subplots(1)
    ax.set_aspect('equal')    
    # add the smlm image
    ax.imshow(image)
#    # add circle of selected region
#    selected=mppatches.Arc((size[0]/2.0,size[1]/2.0),region,region)
#    ax.add_patch(selected)
    #turn of autoscale to stop weird white borders.
    ax.autoscale(False)
    #plot red cross to show where we started. 
    #plot axis are flipped relative to the image.
    if(meanfits is not None):
        ax.plot(meanfits[:,1]+size[1]/2.0,meanfits[:,0]+size[0]/2.0,'ro')
    # Show the image
    
    plt.show()

In [ ]:
#load preselected positions.
MAXPOS=5000
filename=['']*MAXPOS
pointPos=N.zeros((MAXPOS,5))
headers=['']*6
i=0
import csv
with open('lisa-real-data/'+DATASET+'/'+DATASET+'.csv', 'rUb') as csvfile:
    pointreader = csv.reader(csvfile, delimiter=',')
    for row in pointreader:
        if i==0:
            headers=row
        else:
            filename[i-1]=row[0]
            pointPos[i-1]=[float(row[1]),float(row[2]),float(row[3]),float(row[4]),float(row[5])]
        i=i+1
maxpos=i-1
#print filename[:maxpos], pointPos[:maxpos]
print "loaded positions=",maxpos

In [ ]:
#Extract regions around the positions loaded above.
#
#file names in filename[]
#positions of pre-selected points in pointPos[] as
# 0 - number in this file
# 1 - SIM x pos (in um)
# 2 - SIM y pos (in um)
# 3 - STORM X pos (in nm) 
# 4 - STORM Y pos (in nm)
# image possitions are not simply related as they are not aligned. 
i=0
#init arrays of subImages, and good regions
subImage=N.zeros((maxpos,WINDOW*2,WINDOW*2))
#loop for each unique filename
res=N.zeros((maxpos+1))

while i<maxpos:
    fname=filename[i]
#    print fname
    # Load data
#    data = Mrc.open('lisa-real-data/'+str(fname)+'_SIM.tif')
    with tifffile.TiffFile('lisa-real-data/'+DATASET+'/'+str(fname)+'_SIM.tif') as tif:
        #read image data. 
        zStack=tif.asarray()
        #extra tif tags to get resolution. 
        for page in tif:
            for tag in page.tags.values():
                if (tag.name == 'x_resolution'):
                    xres=tag.value
                    res[i]=float(xres[1])/float(xres[0])
#                    print 'res=',res[i]
    
    #Extract intersting regions not reloading each file every time. 
    while(filename[i]==fname and i<maxpos):
        xpos=(int(pointPos[i,1]/res[i]))
        ypos=(int(pointPos[i,2]/res[i]))
        print filename[i],pointPos[i,0],i
        subImage[i]=zStack[ypos-WINDOW:ypos+WINDOW,xpos-WINDOW:xpos+WINDOW]
        i=i+1
        res[i]=res[i-1]


In [ ]:
#### generate fits for SIM toroidal Gaussians
#store centres in fitcentre[]
start=time.time()
#inital guess parameters
#(centreX,centreY,radiusMajor,radiusMinor,angle,width,amp,background) 
#= (20.,28.,9.0,6.5,2.1,4.0,22000.0,400.0)
# default initial guess for Lisa 2016/17 centriol data, in pixels.
initparams=(float(WINDOW),float(WINDOW),6.5,6.5,0.01,4.0,3000.0,50)

outparams=N.zeros((maxpos,len(initparams)))
fitImages=N.zeros((maxpos,WINDOW*2,WINDOW*2))
meanSubImage=N.zeros((maxpos,WINDOW*2,WINDOW*2))
fitcentre=N.zeros((maxpos,2))

for i in range(maxpos):
    #else use our default fit above
    meanSubImage[i,:,:]=subImage[i,:,:]
    (outparams[i,:],fitImages[i,:,:]) = ring_gfit(meanSubImage[i,:,:],
                                                                  initparams,
                                                                  returnfitimage=True) 
        #covert angle back into the range -pi to +pi
    outparams[i,4]=outparams[i,4]%(N.pi)
    if (outparams[i,2]<outparams[i,3]):
        temprad=outparams[i,2]
        outparams[i,2]=outparams[i,3]
        outparams[i,3]=temprad
        outparams[i,4]=outparams[i,4]-(N.pi/2.0)
        
    #covert centres back into real positions. 
    fitcentre[i,0]=pointPos[i,1]+(outparams[i,0]-WINDOW)*res[i]
    fitcentre[i,1]=pointPos[i,2]+(outparams[i,1]-WINDOW)*res[i]

end=time.time()
print "fitting took ",end-start

In [ ]:
#This chunk of code is a fudge to align SIM and LM data. It takes 
# the fitted SIM positions and human (Lisa!) selected LM 
#positions and does a least squares fit with xshift, yshift
#angle and scale.

# find the number of files in this data set and the split points in the relevant arrays
fits=N.zeros((maxpos,4))
initerr = N.zeros((maxpos))
starts= N.zeros((maxpos),dtype=int)
nfiles=0
i=0
while (i<maxpos):
    starts[nfiles]=i
    lastfile=filename[i]
    while ((filename[i]==lastfile) and (i<maxpos)):
        lastfile=filename[i]
        i=i+1
    nfiles=nfiles+1
#final stars needs to be set.
starts[nfiles]=maxpos

#print nfiles, starts

#fit the SMLM data to the centroids perviously calculated...
#if there is only one point in this file just fit XY shift, otherwise fit all 4 params. 
for filenum in range (nfiles):
    
    print filenum, filename[starts[filenum]], starts[filenum+1]-starts[filenum]
    if (starts[filenum+1]-starts[filenum]==1):
        #only one position so can only fit xy shift and fitting is trivial. Set scale to 1 
        #and angle to zero for consistency.
        fit=N.zeros((4))
        fit[:2]=-fitcentre[starts[filenum]]+(pointPos[starts[filenum],3:5]*LMPIXELSIZE)
        fit[2]=1.0
        fit[3]=0.0
        
#        print fits[starts[filenum]]
    #fudge as fits of less than 4 data points dont work as the fit function
    #thinks there are 2 or 3 data points and 4 variables! really each has x and y so there are twice
    #as many data points. 
    #Just duplicate each data point so the fit function thinks its ok. 
    elif ((starts[filenum+1]-starts[filenum])<4):
        dpoints=starts[filenum+1]-starts[filenum]
        tempdata1=N.zeros((dpoints*2,2))
        tempdata2=N.zeros((dpoints*2,2))
        tempdata1[:dpoints]=fitcentre[starts[filenum]:starts[filenum+1]]
        tempdata1[dpoints:]=fitcentre[starts[filenum]:starts[filenum+1]]
        tempdata2[:dpoints]=pointPos[starts[filenum]:starts[filenum+1],3:5]*LMPIXELSIZE
        tempdata2[dpoints:]=pointPos[starts[filenum]:starts[filenum+1],3:5]*LMPIXELSIZE
        fit=shiftfit(tempdata1,tempdata2,(0.001,0.001,1.0,0.001))
        
    else:
        #there are more than 3 positons so do a xy, angle, scale fit
        fit=shiftfit(fitcentre[starts[filenum]:starts[filenum+1]],
                     pointPos[starts[filenum]:starts[filenum+1],3:5]*LMPIXELSIZE,(0.001,0.001,1.0,0.001))

    initerr = N.sum((N.sum((fitcentre[starts[filenum]:starts[filenum+1]]-
                            (pointPos[starts[filenum]:starts[filenum+1],
                               3:5]*LMPIXELSIZE))**2,axis=1)**0.5))
    enderr=N.sum((N.sum((fitcentre[starts[filenum]:starts[filenum+1]]-
                                transPoint(pointPos[starts[filenum]:starts[filenum+1],
                                            3:5]*LMPIXELSIZE,fit))**2,axis=1)**0.5))
    #store fit for every point in this image.
    for i in range(starts[filenum],starts[filenum+1]):
        fits[i]=fit
#        print fitcentre[i]
#        print i,fitcentre[i],lm2simpos(N.array([pointPos[i,3:5]*LMPIXELSIZE]),fits[i])
#    print fit, initerr,enderr

    
#initerr = N.sum((N.sum((fitcentre-(pointPos[:maxpos,3:5]*LMPIXELSIZE))**2,axis=1)**0.5))


#fit=shiftfit(fitcentre,pointPos[:maxpos,3:5],(0.1,0.1,1.0,0.001))

#enderr=N.sum(N.sum((fitcentre-transPoint(pointPos[:maxpos,3:5],fit))**2,axis=1)**0.5)

#print initerr,enderr, fit
             

In [ ]:
#This populates roundlist used below so please run!
#show plots and fit data for every fit created above. 
roundlist=N.zeros((maxpos), dtype=bool)
count =0
#allocate a merged image array for SIM data in red and localisation in green
mergedImage=N.zeros((maxpos,WINDOW*2,WINDOW*2,3))
    
for i in range(maxpos):
    if  ((outparams[i,2]/outparams[i,3]) <1.2):
 #       print outparams[i,0:2]
        roundlist[i]=True
        mergedImage[i,:,:,0]=meanSubImage[i]/meanSubImage[i].max()
        print filename[i],pointPos[i,0], i
#       showplots(i,meanSubImage,fitImages)
#        showplotfit(i,meanSubImage,fitImages,outparams[i,0:2])
        
#        printParams(outparams[i],res[i])
#        print fitcentre[i]
#        print 
#        print
        count=count+1
print "selected images = ",count
    


In [ ]:
print "Sizes of selected Asl rings"
print DATASET
for i in range(maxpos):
    if(roundlist[i]):
        print outparams[i,2]*res[i],outparams[i,3]*res[i]

In [ ]:
#This gets the raw localisation data generates images to match the sim data

#window for SIM data is about 300nm so no point in looking much further.
REGION=500.
#through out "bad" localisations"
ACCURACY=30.
#no longer used as we are reading the SIM pixel size and using that.
#PSIZE=20.
DISTANCE=80.0
#number of pixels matched between SIM subimages and smlm images.
#pixels=int(REGION/PSIZE)*2
pixels=WINDOW*2

smlmimage=N.zeros((maxpos,pixels,pixels))

for i in range(maxpos):
    if (roundlist[i]):
        #pixel size determined from sim image pixel size as they are not fixed
        psize=res[i]
        print i,filename[i]
        #show images around the pointPos selected region as this is 
        #used in the sim subimages. 
        trpoint=sim2lmpos(N.array([pointPos[i,1:3]]),fits[i])*1000.
        #print trpoint
        locData=N.loadtxt('lisa-real-data/'+DATASET+'/'+str(filename[i])+'_LM.txt')
        temppointlist=filterPoints(trpoint[0,1],trpoint[0,0],REGION,ACCURACY,locData)/1000.0
        #plt.scatter(-temppointlist[:,2],temppointlist[:,1],s=temppointlist[:,3]*1)
        #We are going to plot in sim space so transofrm our zero pos and data into
        #sim image space to plot the two image ontop of each other.
        trpoint=lm2simpos(trpoint/1000.,fits[i])
        #remeber to transpose X-Y before transformation
        templist=N.zeros((len(temppointlist),2))
        templist[:,0]=temppointlist[:,2]
        templist[:,1]=temppointlist[:,1]
        poslist=lm2simpos(templist,fits[i])
        
        for j in range(len(temppointlist)):
            #note flipped X-Y values in poslist to plot on the correct axis
            smlmimage[i]=smlmimage[i]+gaussian2d((poslist[j,0]-trpoint[0,0])/psize+WINDOW,
                                   (((poslist[j,1]-trpoint[0,1])/psize)+WINDOW),
                                   temppointlist[j,3]/psize,
                                   temppointlist[j,4]/psize,smlmimage[i].shape)
    
    
#    start=(((trpoint[0,0]-bestcentre[i,0])+centre),
#        ((pointPos[i,3]*LMPIXELSIZE*1000.-bestcentre[i,1])+centre))
        region=(DISTANCE*2.)/psize
       # plotsmlmregion(smlmimage,start,region)
        mergedImage[i,:,:,1]=smlmimage[i]/smlmimage[i].max()
        #print "centres=",(fitcentre[i]-pointPos[i,1:3])/res[i]
        showmerged(mergedImage[i],
                   simpos=([0,0]),
                   simfit=(((fitcentre[i,0]-pointPos[i,1])/res[i]),
                           (fitcentre[i,1]-pointPos[i,2])/res[i])
                  # lmfit=(((bestcentre[i,0]/1000.0-pointPos[i,1])/res[i]),
                   #        (bestcentre[i,1]/1000.0-pointPos[i,2])/res[i]),
                    )

#            fitcentre[i,0]=pointPos[i,1]+(outparams[i,0]-WINDOW)*res[i]
#    fitcentre[i,1]=pointPos[i,2]+(outparams[i,1]-WINDOW)*res[i]

        

In [ ]:
#this finds the "middle" of the centriols.
#starts from the fitted SIM points, as offset by the hnad selected LM points
#takes a weighted mean position and iterates until either 50 loops or
#the mean position converges. 
#

#this uses the info dediced above to find good starting points for 
#the localisation peaks. Optomises the data selection with weigthed averages
#
#fastSPDM spec to date....
# spec to date.
# 0) ?
# 1 - X pos nm
# 2 - Y pos nm
# 3 - Loc accurancy
# 4 - Loc accurancy
# 5 - sigma of PSF
# 6 - sigma of PSF
# 7 - number of detected photons
# 8 - frame number


#Loop through locData and find localisations near the defined points. 
#radial serach distance from wheer we start.
DISTANCE=100.0
#threshold accurancy. Things less accurate (ie higher number)
#are discarded.
ACCURACY=100.0
#A threshold to dispose of bad inital guesses where there are not at 
#lest N localisations within the DISTANCE param
NUMBER=30

#for plots
WINDOW=15
psize=20.
#this shows the fitting steps but makes it much slower.
showsteps=True
#showsteps=False


regions=N.zeros((maxpos,2))
pointlist=N.empty((maxpos,500,12),dtype=float)
#store xy and weights shifted to the best fit centre. 
shiftedpointlist=N.empty((maxpos,500,3),dtype=float)
maxpoint=N.zeros((maxpos),dtype=int)
bestcentre=N.zeros((maxpos,2))
meanfits=N.zeros((maxpos,50,2))
rads=N.zeros((1))

i=0
#loop over all centriols
while i<maxpos:
    # Get and open localisation text file.
    fname=filename[i]
    locData=N.loadtxt('lisa-real-data/'+DATASET+'/'+str(fname)+'_LM.txt')
 #   print len(locData)
    #loop though centriols in  this file so we dont need to reload.
    while(filename[i]==fname and i<maxpos):
        #find localisations near the suggested point
        # localisation positions are switched in XY!
        regions[i]=sim2lmpos(N.array([fitcentre[i]]),fits[i])*1000.
        #regions[i,0]=centre[0]
        #regions[i,1]=centre[1]
#        print i,filename[i],regions[i]
        #find points within DISTANCE and populate pointlist[i]
        maxpoint[i]=len(filterPoints(regions[i,1],regions[i,0],DISTANCE,ACCURACY,locData))
        pointlist[i,:maxpoint[i],:]= filterPoints(regions[i,1],regions[i,0],DISTANCE,ACCURACY,locData)
        #find new mean positions
        #start with a simple mean
        xmean=pointlist[i,:maxpoint[i],1].mean()
        ymean=pointlist[i,:maxpoint[i],2].mean()
        oldmeanX=0.0
        oldmeanY=0.0
        loops=0
        #loop to optomise centroid, based on new points entering region as we move
        #the centre.
        while(((oldmeanX != xmean) or (oldmeanY != ymean)) and loops<49):
            oldmeanX=xmean
            oldmeanY=ymean
           
            #replace pointlist with values around oldmeanX and oldmeanY
            maxpoint[i]=len(filterPoints(oldmeanX,oldmeanY,DISTANCE,ACCURACY,locData))
            pointlist[i,:maxpoint[i],:]=filterPoints(oldmeanX,oldmeanY,DISTANCE,ACCURACY,locData)
            #calculate the weights as 1/(RMS aevarge location accurancy)
            weights=(1/(((pointlist[i,:maxpoint[i],3]**2)+(pointlist[i,:maxpoint[i],4]**2))**0.5))
            #weigthed mean positions in X and Y
            (xmean,xvar)=weightedMean(pointlist[i,:maxpoint[i],1],weights)
            (ymean,xvar)=weightedMean(pointlist[i,:maxpoint[i],2],weights)
            meanX=pointlist[i,:maxpoint[i],1].mean()
            meanY=pointlist[i,:maxpoint[i],2].mean()
            meanfits[i,loops,:]=[xmean,ymean]
            loops=loops+1
#            print "Mean XY=",meanX,meanY
#            print "Weighted Means=",xmean,ymean
#            print "points=", maxpoint[i]
            if (showsteps):
                tempimage=N.zeros((WINDOW*2,WINDOW*2))
                for j in range(len(pointlist)):
                    #note flipped X-Y values in poslist to plot on the correct axis
                    tempimage=tempimage+gaussian2d((((pointlist[i,j,2]-ymean)/psize)+WINDOW),
                                       (((pointlist[i,j,1]-xmean)/psize)+WINDOW),
                                       pointlist[i,j,3]/psize,
                                       pointlist[i,j,4]/psize,tempimage.shape)
                showimage(tempimage,(meanfits[i,:loops,:]-[[xmean,ymean]])/psize)
                print "Mean XY=",meanX,meanY
                print "Weighted Means=",xmean,ymean
                print "points=", maxpoint[i]
                print
        
        #calculate distance from centroid,
        radii=checkdist(xmean,ymean,pointlist[i,:maxpoint[i],1],pointlist[i,:maxpoint[i],2])
        #plots scatter of all the found localisations...
        #plt.scatter(pointlist[i,:maxpoint[i],2],-pointlist[i,:maxpoint[i],1],s=0.01)
#        print radii.mean(), radii.std(), xmean, ymean, loops
        regions[i]=[xmean,ymean]
        #bestcentre is XY in LM text files, flipped relative to SIM images!
        bestcentre[i]=regions[i]
        #find locations and weights for weighted fitting from localisation precision
#        locations=pointlist[i,:maxpoint[i],1:3] 
        weights=(1/(((pointlist[i,:maxpoint[i],3]**2)+(pointlist[i,:maxpoint[i],4]**2))))
        #use a weighted variance fitting function to optomise.
#        bestcentre[i]=fitweightedcentre(locations,weights,regions[i]) 
        
        (wmeanradii,wvarradii)=weightedMean(radii,weights)
        rads=N.append(rads,radii,axis=0)
#        print i,wmeanradii,wvarradii**0.5,bestcentre[i,0],bestcentre[i,1],maxpoint[i]
#        print "shift=",(sim2lmpos(N.array([fitcentre[i]]),fits[i])*1000.)-[bestcentre[i,1],bestcentre[i,0]]
        shiftedpointlist[i,:maxpoint[i],0:2]=pointlist[i,:maxpoint[i],1:3]-bestcentre[i]
        shiftedpointlist[i,:maxpoint[i],2]=weights
        i=i+1
plt.hist(rads)

In [ ]:
hist=N.array ([ 178.,  449.,  640.,  808.,  883.,  729.,  712.,  629.,  492.,  407.])
histdensity=N.zeros((hist.shape))
for i in range(10):
    histdensity[i]=hist[i]/((i+0.5)*10.)
plt.plot(histdensity)
plt.show()
plt.plot(hist)
print histdensity


In [ ]:
#cell above finds the centres of the "best" local region, now try to reselected 
#based on these centres but with a smaller region included, maybe 50 nm rather than
#100 (ie 1/4 as much area) then optomised the radius based on this. 

#General rule of thumb, set the DISTANCE to 2x the peak in the radii above.

DISTANCE=90.0
#threshold accurancy. Things less accurate (ie higher number)
#are discarded.
ACCURACY=30.0
weightedbestcentre=N.zeros((maxpos,2))
fittedrads=N.zeros((1))

lmregion=DISTANCE

i=0
#loop over all centriols
while i<maxpos:
    # Get and open localisation text file.
    fname=filename[i]
    locData=N.loadtxt('lisa-real-data/'+DATASET+'/'+str(fname)+'_LM.txt')
 #   print len(locData)
    #loop though centriols in  this file so we dont need to reload.
    while(filename[i]==fname and i<maxpos):
        #only do the final processing on the "round" ones. 
        if(roundlist[i]):
            #find localisations near the suggested point
            # localisation positions are switched in XY!
            maxpoint[i]=len(filterPoints(bestcentre[i,0],bestcentre[i,1],DISTANCE,ACCURACY,locData))
            pointlist[i,:maxpoint[i],:]=filterPoints(bestcentre[i,0],bestcentre[i,1],DISTANCE,ACCURACY,locData)
            #calculate the weights as 1/(RMS aevarge location accurancy)
            weights=(1/(((pointlist[i,:maxpoint[i],3]**2)+(pointlist[i,:maxpoint[i],4]**2))**0.5))
            #find locations and weights for weighted fitting from localisation precision
            locations=pointlist[i,:maxpoint[i],1:3] 
            weights=(1/(((pointlist[i,:maxpoint[i],3]**2)+(pointlist[i,:maxpoint[i],4]**2))))
            #use a weighted variance fitting function to optomise.
            weightedbestcentre[i]=fitweightedcentre(locations,weights,regions[i]) 
            radii=checkdist(weightedbestcentre[i,0],weightedbestcentre[i,1],pointlist[i,:maxpoint[i],1],
                            pointlist[i,:maxpoint[i],2]) 
            (wmeanradii,wvarradii)=weightedMean(radii,weights)
#            print i,wmeanradii,wvarradii**0.5,bestcentre[i,0],bestcentre[i,1],maxpoint[i]
#            print "shift=",(sim2lmpos(N.array([fitcentre[i]]),
#                                      fits[i])*1000.)-[weightedbestcentre[i,1],weightedbestcentre[i,0]]
            shiftedpointlist[i,:maxpoint[i],0:2]=pointlist[i,:maxpoint[i],1:3]-weightedbestcentre[i]
            shiftedpointlist[i,:maxpoint[i],2]=weights
            fittedrads=N.append(fittedrads,radii,axis=0)
        i=i+1
plt.hist(fittedrads)

In [ ]:

hist=N.array ([  57.,  232.,  341.,  500.,  451.,  400.,  337.,  193.,   99.,   17.])
histdensity=N.zeros((hist.shape))
for i in range(10):
    histdensity[i]=hist[i]/((i+0.5)*10.)
plt.plot(histdensity)
plt.show()
plt.plot(hist)
print histdensity


In [ ]:
#This plots the merged SIM/LM images with SIM in red and LM is green
#red spot for inital SIM guess
#green spot for centre of SIM fit/initial LM guess
# black spot for centre of lm fit
# black circle bounding the lm region used for fitting

for i in range (maxpos):
    if (roundlist[i]):
        #remember to flip the XY axis from lm to sim pos
        lmfit=lm2simpos(N.array([[bestcentre[i,1]/1000.0,
                                    bestcentre[i,0]/1000.0]]),fits[i])
        print i
        showmerged(mergedImage[i],
                   simpos=([0,0]),
                   simfit=(((fitcentre[i,0]-pointPos[i,1])/res[i]),
                           (fitcentre[i,1]-pointPos[i,2])/res[i]),
                    lmfit=(((lmfit[0,0]-pointPos[i,1])/res[i]),
                           (lmfit[0,1]-pointPos[i,2])/res[i]),
                   region=lmregion*2.0/(res[i]*1000.)
                    )

In [ ]:
i=17

lmfit=lm2simpos(N.array([[bestcentre[i,1]/1000.0,
                        bestcentre[i,0]/1000.0]]),fits[i])
showmerged(mergedImage[i],
                   simpos=([0,0]),
                   simfit=(((fitcentre[i,0]-pointPos[i,1])/res[i]),
                           (fitcentre[i,1]-pointPos[i,2])/res[i]),
                    lmfit=(((lmfit[0,0]-pointPos[i,1])/res[i]),
                           (lmfit[0,1]-pointPos[i,2])/res[i]),
                   region=lmregion*2.0/(res[i]*1000.)
                    )

In [ ]:
print bestcentre[i]

In [ ]:
fitcentre[i]/res[i]

In [ ]:
#generate a very high res SMLM image for publication
REGION=120.
#through out "bad" localisations"
ACCURACY=30.
#no longer used as we are reading the SIM pixel size and using that.
#PSIZE=20.
DISTANCE=80.0
#number of pixels matched between SIM subimages and smlm images.
#pixels=int(REGION/PSIZE)*2
WIN=300
pixels=WIN*2

pubsmlmimage=N.zeros((pixels,pixels))

i=17
psize=0.001
print i,filename[i]
#show images around the pointPos selected region as this is 
#used in the sim subimages. 
#lmfit=lm2simpos(N.array([[bestcentre[i,1]/1000.0,
#                        bestcentre[i,0]/1000.0]]),fits[i])
trpoint=sim2lmpos(N.array([pointPos[i,1:3]]),fits[i])*1000.

#print trpoint
locData=N.loadtxt('lisa-real-data/'+DATASET+'/'+str(filename[i])+'_LM.txt')
temppointlist=filterPoints(bestcentre[i,0],bestcentre[i,1],REGION,ACCURACY,locData)/1000.0
#plt.scatter(-temppointlist[:,2],temppointlist[:,1],s=temppointlist[:,3]*1)
#We are going to plot in sim space so transofrm our zero pos and data into
#sim image space to plot the two image ontop of each other.
trpoint=lm2simpos(trpoint/1000.,fits[i])
#remeber to transpose X-Y before transformation
templist=N.zeros((len(temppointlist),2))
templist[:,0]=temppointlist[:,2]
templist[:,1]=temppointlist[:,1]
poslist=lm2simpos(templist,fits[i])

#finally shift by the correct amount to match the diff between subImage and 
#the real fit centre
#shift=(pointPos[i,1:3]-fitcentre[i])*1000
shift=[0,0]
for j in range(len(temppointlist)):
    #note flipped X-Y values in poslist to plot on the correct axis
    #and shift to match to the fitted centre in the original image.
    pubsmlmimage=pubsmlmimage+gaussian2d((poslist[j,0]-trpoint[0,0])/psize+WIN-int(shift[0]),
                                   (((poslist[j,1]-trpoint[0,1])/psize)+WIN-int(shift[1])),
                                   (temppointlist[j,3])/psize,
                                   (temppointlist[j,4])/psize,pubsmlmimage.shape)
    
plt.imshow(pubsmlmimage)



In [ ]:
print shift

In [ ]:
import scipy.misc


In [ ]:
for i in range(maxpos):
    if roundlist[i]:
        lmfit=lm2simpos(N.array([[bestcentre[i,1]/1000.0,
                                    bestcentre[i,0]/1000.0]]),fits[i])
        print i,(lmfit-fitcentre[i])*1000

In [ ]:
i=17
scale=res[i]/0.001
size = mergedImage.shape[1:3]
pubImage=N.zeros((int(size[0]*scale),int(size[1]*scale),3),dtype=int)

fig, ax = plt.subplots(figsize=(10,10),dpi=100)

pubImage[:,:,0]=254-scipy.misc.imresize(mergedImage[4,:,:,0],scale,interp='bicubic')*(253./255.)
pubImage[:,:,1]=250-(pubsmlmimage*(240.0/pubsmlmimage.max()))
#print pubImage[:,:,0].max(),pubImage[:,:,0].min()
plt.axis('off')
ax.imshow(pubImage)
#plt.plot(pubImage[300,:,0])
#plt.plot( pubImage[200,:,0])
#print pubImage[300,:,1]

In [ ]:
#display a merged image of the above centriols.
#size in nm
IMSIZE=150.
#pixelszie in nm
PIXELSIZE=1.0
pixels=IMSIZE/PIXELSIZE
tempimage=N.zeros((int(pixels)*2,int(pixels)*2))
count=0
for i in range(maxpos):
    if roundlist[i]:
        for j in range(maxpoint[i]):
            count=count+1
            #note flipped X-Y values in poslist to plot on the correct axis
            tempimage=tempimage+gaussian2d((((shiftedpointlist[i,j,1])/PIXELSIZE)+pixels),
                                       (((shiftedpointlist[i,j,0])/PIXELSIZE)+pixels),
                                       pointlist[i,j,3]/PIXELSIZE/2.,
                                       pointlist[i,j,4]/PIXELSIZE/2.,tempimage.shape)
plt.imshow(tempimage,clim=(0.,tempimage.max()*1.15))
plt.colorbar()
print count, tempimage.max(),tempimage.min()


In [ ]:
#export summed image File
outfile=open(DATASET+'merge.npy','w')
N.save(outfile,tempimage)
tempimage.shape

In [ ]:
#This is the ANSWER!
#print mean radii localisations of only the round centriols. 
for i in range (maxpos):
    if roundlist[i]:
        #plt.scatter(shiftedpointlist[i,:maxpoint[i],0],
        #            shiftedpointlist[i,:maxpoint[i],1],
        #            s=shiftedpointlist[i,:maxpoint[i],2]*30)
        locations=pointlist[i,:maxpoint[i],1:3] 
        weights=(1/(((pointlist[i,:maxpoint[i],3]**2)+
                     (pointlist[i,:maxpoint[i],4]**2))**0.5))
        #use a weights variance fitting function to optomise.
        radii=checkdist(bestcentre[i,0],bestcentre[i,1],
                        pointlist[i,:maxpoint[i],1],
                        pointlist[i,:maxpoint[i],2]) 
        (wmeanradii,wvarradii)=weightedMean(radii,weights)
        print "weighted means", i,wmeanradii,wvarradii**0.5,bestcentre[i,0],bestcentre[i,1],maxpoint[i]
        
        
        

In [ ]:
#this is a test function that prints the lm regions around the 
#fitted centriol locations. Useful diagnostic.

i=1
REGION=500.
PSIZE=2.
pixels=int(REGION/PSIZE)*2

for i in range(maxpos):
    print filename[i],pointPos[i],bestcentre[i,:]/(LMPIXELSIZE*1000.)
    locData=N.loadtxt('lisa-real-data/'+DATASET+'/'+str(filename[i])+'_LM.txt')
    temppointlist=filterPoints(bestcentre[i,0],bestcentre[i,1],REGION,ACCURACY,locData)
    #plt.scatter(-temppointlist[:,2],temppointlist[:,1],s=temppointlist[:,3]*1)

    smlmimage=N.zeros((pixels,pixels))
    for j in range(len(temppointlist)):
        smlmimage=smlmimage+gaussian2d((temppointlist[j,1]-bestcentre[i,0]+REGION)/PSIZE,
                                   (temppointlist[j,2]-bestcentre[i,1]+REGION)/PSIZE,
                                   temppointlist[j,3]/PSIZE,
                                   temppointlist[j,4]/PSIZE,smlmimage.shape)
    
    
    centre=REGION/PSIZE
    start=(((pointPos[i,4]*LMPIXELSIZE*1000.-bestcentre[i,0])+centre),
        ((pointPos[i,3]*LMPIXELSIZE*1000.-bestcentre[i,1])+centre))
    region=(DISTANCE*2.)/PSIZE
    plotsmlmregion(smlmimage,start,region)